In [1]:
import pandas as pd
import datetime
import copy

# time = datetime.datetime.now()
# time_now = time.strftime('%Y-%m-%d %H:%M:%S')
xlsx_file_name = 'original'
xlsx_file = f'{xlsx_file_name}.xlsx'
xlsx_file_start = f'{xlsx_file}_start.xlsx'
export_file_name = f'{xlsx_file}_final.xlsx'  # * 出力ファイル名

# extract target_connect_id and make it List
df = pd.read_excel(f'./{xlsx_file}', header=0)
print('df: ', df)

extract_col = ['最終納品先店舗名', '販売日', '商品名', '販売単価', '出荷確定数', '生産者名', '産地市町村名', 'JANコード', '商品入数']

df = df[extract_col]
df = df.sort_values(by=['最終納品先店舗名', '商品名'])
rename_col = {'販売単価': '掲載単価', '産地市町村名': '産地（都道府県）', '数量': '入数'}
df = df.rename(columns=rename_col)
df['規格'] = df['商品入数'].str.cat(df['単位'])
extract_col = ['ID', '商品名', '出荷確定数', '規格', 'JANコード', '掲載単価', '販売日']

# * df分割
list = {
    'minamisuna': '南砂店',
    'makuharishintoshin': '幕張新都心店',
    'asahichuo': '旭中央店',
    'shinonome': '東雲店',
    'kaihinmakuhari': '海浜幕張店',
    'kasai': '葛西店',
    'chousi': '銚子店',
    'kamatori': '鎌取店',
    'tateyama': '館山店',
    'kamogawa': '鴨川店',
}

for k, v in list.items():
    shop_list = df['最終納品先店舗名'].to_list()
    if v in shop_list:
        df_shop = df.groupby('最終納品先店舗名').get_group(v)
    else:
        continue
    df_shop = df_shop[extract_col]
    print('df_shop: ', df_shop)

    product_name = [k for k in dict.fromkeys(df_shop['商品名']).keys()]

    # todo 年を削除して、月日にする
    df_shop = df_shop.sort_values('販売日')
    days = [k.strftime('%-m/%-d') for k in dict.fromkeys(df_shop['販売日']).keys()]  # todo For Windows %#m/%#d

    order_list = pd.DataFrame(columns=days, index=product_name)
    order_list = order_list.fillna(0)
    for i, v in df_shop.iterrows():
        order_list.at[v[1], v[0].strftime('%-m/%-d')] = v[2]  # todo For Windows %#m/%#d

    # print('order_list: ', order_list)
    order_list.to_excel(f'./{k}.xlsx', index_label='商品名')  # * index_labelは、product_idにする


df:                           需給取引No   利用企業コード           利用企業名    得意先コード  \
0      JK2023022800009014-1-1-1  20000201  イオンリテール株式会社ＧＭＳ  20000201   
1      JK2023022800009014-1-8-2  20000201  イオンリテール株式会社ＧＭＳ  20000201   
2     JK2023030200009152-1-15-3  20000201  イオンリテール株式会社ＧＭＳ  20000201   
3      JK2023030200009152-1-8-2  20000201  イオンリテール株式会社ＧＭＳ  20000201   
4     JK2023022800009057-1-16-6  20000201  イオンリテール株式会社ＧＭＳ  20000201   
..                          ...       ...             ...       ...   
376   JK2023030200009135-1-15-3  20000201  イオンリテール株式会社ＧＭＳ  20000201   
377    JK2023030200009135-1-1-1  20000201  イオンリテール株式会社ＧＭＳ  20000201   
378   JK2023030200009135-1-17-9  20000201  イオンリテール株式会社ＧＭＳ  20000201   
379   JK2023030200009135-1-4-10  20000201  イオンリテール株式会社ＧＭＳ  20000201   
380  JK2023030200009135-1-18-12  20000201  イオンリテール株式会社ＧＭＳ  20000201   

               得意先名  最終納品先店舗コード 最終納品先店舗名        出荷日  最終納品先納品日        販売日  ...  \
0    イオンリテール株式会社ＧＭＳ         608      銚子店 2023-03-08       NaN

KeyError: '商品入数'